## Preprocess Flight Delay Data
This only needs to be run once. It requires downloading all of the datasets from the Bureau of Transportation. To recreate, follow steps: 
- visit the [Bureau of Transportation](https://www.transtats.bts.gov/databases.asp?Z1qr_VQ=E&Z1qr_Qr5p=N8vn6v10&f7owrp6_VQF=D)
- select Airline Performance Data
- select download for Reporting Carrier On-Time Performance (1987-Present)
- select relevant features from GUI. See relevant_columns below

### Preprocess data steps
1. Trim data, keeping only relevant columns
2. Balance Data, keeping 10k samples from each month and class (Delayed, Not Delayed)
3. Join years and months and save to single csv file flight_data.csv

> |Total row count is 713,664. Slightly under 720k, as some months didn't have 10k samples for each class.

In [26]:
import pandas as pd


YEARS= ["2020", "2021", "2022"]
MONTHS =[ "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
base_filename = "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_"
base_file_folder = "On_Time_Reporting_Carrier_On_Time_Performance_1987_present_"
src_filepath = "~/Downloads/"
dest_filepath = "../data/"

RECORDS_TO_KEEP_FROM_EACH_DATASET = 10000

In [3]:
data_files = []
for year in YEARS:
    for month in MONTHS:
        filename = base_filename + year + "_" + month + ".csv"
        folder = base_file_folder + year + "_" + month
        data_files.append(src_filepath + folder + "/" + filename)

In [4]:
print(data_files)

['~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_1/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_1.csv', '~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_2/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_2.csv', '~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_3/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_3.csv', '~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_4/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_4.csv', '~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_5/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_5.csv', '~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_6/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_6.csv', '~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_7/On

In [5]:
relevant_columns = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', "Tail_Number", "Origin", "Dest", "DepTime", "DepDelay",
        "ArrTime", "ArrDelay", "ArrDel15", "Cancelled", "CancellationCode",
       "Diverted", "AirTime", "Flights", "Distance", "CarrierDelay", "WeatherDelay","NASDelay", "SecurityDelay", "LateAircraftDelay"]
print(len(relevant_columns))

26


In [6]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    # Step 1 -- trim
    df_trimmed = df[relevant_columns]
    # Step 2 -- balance
    # Find delayed flights, indicated by ArrDel15 == 1
    df_delayed = df_trimmed[df_trimmed["ArrDel15"] == 1]
    keep_amount = min(len(df_delayed), RECORDS_TO_KEEP_FROM_EACH_DATASET)
    print(f"there are {len(df_delayed)} flight records that were delayed. randomly sampling {keep_amount}")
    df_delayed_sample = df_delayed.sample(n=keep_amount, random_state=1)

    df_not_delayed = df_trimmed[df_trimmed["ArrDel15"] == 0] 
    print(f"there are {len(df_not_delayed)} flight records that weren't delayed, sampling {keep_amount} of them")

    df_non_delayed_sample = df_not_delayed.sample(n=keep_amount, random_state=1)

    return pd.concat([df_delayed_sample, df_non_delayed_sample])


In [7]:
preprocessed_data = []
for file in data_files:
    df = pd.read_csv(file)
    print(f"processing {file}")
    preprocessed_data.append(preprocess_data(df))
    
all_flights_df = pd.concat(preprocessed_data)

/var/folders/x5/cgpxmq_11ys090v206spbw7w0000gn/T/ipykernel_48396/749170752.py:3: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


processing ~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_1/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_1.csv
there are 82285 flight records that were delayed. randomly sampling 10000
there are 516983 flight records that weren't delayed, sampling 10000 of them


/var/folders/x5/cgpxmq_11ys090v206spbw7w0000gn/T/ipykernel_48396/749170752.py:3: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


processing ~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_2/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_2.csv
there are 84616 flight records that were delayed. randomly sampling 10000
there are 483460 flight records that weren't delayed, sampling 10000 of them


/var/folders/x5/cgpxmq_11ys090v206spbw7w0000gn/T/ipykernel_48396/749170752.py:3: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


...

processing ~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2022_10/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_10.csv
there are 90057 flight records that were delayed. randomly sampling 10000
there are 476435 flight records that weren't delayed, sampling 10000 of them


/var/folders/x5/cgpxmq_11ys090v206spbw7w0000gn/T/ipykernel_48396/749170752.py:3: DtypeWarning: Columns (76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


processing ~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2022_11/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_11.csv
there are 102278 flight records that were delayed. randomly sampling 10000
there are 436921 flight records that weren't delayed, sampling 10000 of them


/var/folders/x5/cgpxmq_11ys090v206spbw7w0000gn/T/ipykernel_48396/749170752.py:3: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


processing ~/Downloads/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2022_12/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_12.csv
there are 142407 flight records that were delayed. randomly sampling 10000
there are 382808 flight records that weren't delayed, sampling 10000 of them


In [8]:
all_flights_df.describe()

,Year,Quarter,Month,DayofMonth,DayOfWeek,DepTime,DepDelay,ArrTime,ArrDelay,ArrDel15,Cancelled,Diverted,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,713664.000000,713664.000000,713664.000000,713664.000000,713664.00000,713664.00000,713664.000000,713664.000000,713664.000000,713664.0,713664.0,713664.0,713664.000000,713664.0,713664.000000,356831.000000,356831.000000,356831.000000,356831.000000,356831.000000
mean,2021.008878,2.504439,6.516013,15.747531,4.03537,1386.12710,28.835928,1506.231169,26.835046,0.5,0.0,0.0,114.219529,1.0,816.095734,27.084687,4.367916,11.717600,0.188613,20.829793
std,0.814635,1.121989,3.462869,8.746131,2.00945,492.44317,75.004397,546.612997,75.848376,0.5,0.0,0.0,69.325232,0.0,576.604762,73.908028,34.414985,29.410674,3.905450,51.973683
min,2020.000000,1.000000,1.000000,1.000000,1.00000,1.00000,-83.000000,1.000000,-87.000000,0.0,0.0,0.0,8.000000,1.0,29.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2020.000000,1.000000,3.000000,8.000000,2.00000,1004.00000,-4.000000,1131.000000,-11.000000,0.0,0.0,0.0,63.000000,1.0,391.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2021.000000,3.000000,7.000000,16.000000,4.00000,1411.00000,2.000000,1550.000000,14.500000,0.5,0.0,0.0,99.000000,1.0,679.000000,6.000000,0.000000,0.000000,0.000000,0.000000
75%,2022.000000,4.000000,10.000000,23.000000,6.00000,1805.00000,37.000000,1940.000000,37.000000,1.0,0.0,0.0,145.000000,1.0,1055.000000,26.000000,0.000000,16.000000,0.000000,22.000000
max,2022.000000,4.000000,12.000000,31.000000,7.00000,2400.00000,2650.000000,2400.000000,2647.000000,1.0,0.0,0.0,727.000000,1.0,5812.000000,2647.000000,1805.000000,1642.000000,789.000000,2093.000000


In [9]:
all_flights_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 713664 entries, 559890 to 409349
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Year               713664 non-null  int64  
 1   Quarter            713664 non-null  int64  
 2   Month              713664 non-null  int64  
 3   DayofMonth         713664 non-null  int64  
 4   DayOfWeek          713664 non-null  int64  
 5   FlightDate         713664 non-null  object 
 6   Reporting_Airline  713664 non-null  object 
 7   Tail_Number        713664 non-null  object 
 8   Origin             713664 non-null  object 
 9   Dest               713664 non-null  object 
 10  DepTime            713664 non-null  float64
 11  DepDelay           713664 non-null  float64
 12  ArrTime            713664 non-null  float64
 13  ArrDelay           713664 non-null  float64
 14  ArrDel15           713664 non-null  float64
 15  Cancelled          713664 non-null  float64
 1

In [27]:
output_file = "flight_data.csv"
print(f"writing to {output_file}")
all_flights_df.to_csv(dest_filepath + output_file, index=False)

writing to flight_data.csv
